<a href="https://colab.research.google.com/github/zying0113/Quantitative_Investment_Analysis/blob/main/Investment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 資料庫使用

## 安裝FinMind ＆ 引用套件

In [1]:
pip install FinMind

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 748.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.1/147.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 7.3 MB/s eta 0:00:00
  Created wheel for ta: filename=ta-0.5.25-py3-none-any.whl size=24878 sha256=8e2c89d06e210dc96a6f228e85a6d40da152b729ff91b2182e42fdb5d343

In [11]:
#基本套件
import pandas as pd
import numpy as np
import datetime
from copy import deepcopy
import re
from functools import reduce
from collections import defaultdict, OrderedDict
from tqdm import trange, tqdm


# 繪圖工具
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly import subplots


#FinMind
from FinMind.data import DataLoader

## 撈取資料

In [12]:
stock_no = '2731'
start, end ='2020-01-01', '2023-05-03'
principal = 500000 ##本金

d1 = DataLoader()
stock = d1.taiwan_stock_daily(stock_id=stock_no, start_date=start, end_date=end)
stock.rename(columns={'date':'mdate', 'open':'open_d', 
                        'max':'high_d', 'min':'low_d',
                        'close':'close_d', 'Trading_Volume':'volume', }, inplace = True)
stock.drop(columns=['stock_id','Trading_money','spread','Trading_turnover'], inplace = True)
stock['mdate'] = pd.to_datetime(stock['mdate']) # 轉換日期格式('文字'轉'時間')

# 大盤
d2 = DataLoader()
market = d2.taiwan_stock_daily(stock_id='TAIEX', start_date=start, end_date=end)
market.rename(columns={'date':'mdate', 'open':'open_m', 
                        'max':'high_m', 'min':'low_m',
                        'close':'close_m', 'Trading_Volume':'volume_m', }, inplace = True)
market.drop(columns=['stock_id','Trading_money','spread','Trading_turnover'], inplace = True)
market['mdate'] = pd.to_datetime(market['mdate'])

In [ ]:
stock

,mdate,volume,open_d,high_d,low_d,close_d
0,2020-01-02,176278,85.3,85.3,83.9,84.1
1,2020-01-03,251176,84.0,84.0,81.6,82.5
2,2020-01-06,98132,82.5,83.2,82.0,82.6
3,2020-01-07,221202,82.8,84.6,82.8,82.8
4,2020-01-08,112091,82.7,83.0,82.1,83.0
...,...,...,...,...,...,...
803,2023-04-26,1800073,170.0,173.0,168.0,173.0
804,2023-04-27,6525306,170.0,171.0,161.0,171.0
805,2023-04-28,6677341,172.0,181.5,170.0,176.5
806,2023-05-02,3136432,178.5,181.5,174.0,176.0


# 技術型指標

In [2]:
def MACD(DF, a = 12, b =26, c =9):
    '''
    簡單移動平均線: rolling
    指數移動平均線: ewm
    '''
    df = DF.copy()
    df["EMA_Fast"] = df['open_d'].ewm(span = a, min_periods = a).mean()
    df["EMA_Slow"] = df['open_d'].ewm(span = b, min_periods = b).mean()
    df["DIF"] = df["EMA_Fast"] - df["EMA_Slow"]
    df["MACD"] = df['DIF'].ewm(span = c, min_periods = c).mean()
    
    return df

In [3]:
def BollingerBand(DF, n=20):
    '''
    標準差: std
    '''
    df = DF.copy()
    df['SMA'] = df['close_d'].rolling(window = n).mean()
    df['BBu'] = df['SMA'] + 2*df['close_d'].rolling(window = n).std(ddof=0)
    df['BBd'] = df['SMA'] - 2*df['close_d'].rolling(window = n).std(ddof=0)
    df['BB_width'] = df['BBu'] - df['BBd']
    #df.dropna(inplace = True)
    
    return df

RSI指標的計算流程如下:
1.   依據隔日價差，計算 n日內平均的漲跌幅
2.   計算相對強度(RS)：n日內漲幅平均值 / n日內跌幅平均值
3.   計算相對強弱指標(RSI)：相對強度(RS) / (1 + 相對強度(RS)) × 100

In [4]:
def RSI(DF, n=14):
    '''
    n日漲幅平均值 = n日內上漲日總上漲幅度加總 ÷ n
    n日跌幅平均值 = n日內下跌日總下跌幅度加總 ÷ n
    '''
    df = DF.copy()
    df['daliy_change'] = df['close_d'] - df['close_d'].shift(1)
    df['dUp'] = np.where(df['daliy_change'] >= 0, df['daliy_change'], 0)
    df['dDown'] = np.where(df['daliy_change'] < 0, -df['daliy_change'], 0)
    avg_dUp = []
    avg_dDown = []
    dUp = df['dUp'].tolist()
    dDown = df['dDown'].tolist()
    
    for i in range(len(df)):
        if i < n:
            avg_dUp.append(0)
            avg_dDown.append(0)
        elif i == n:
            avg_dUp.append(df['dUp'].ewm(span = n).mean()[n])
            avg_dDown.append(df['dDown'].ewm(span = n).mean()[n])
        else:
            avg_dUp.append(((n-1)*avg_dUp[i-1] + dUp[i])/n)
            avg_dDown.append(((n-1)*avg_dDown[i-1] + dDown[i])/n)
    
    df['avg_dUp'] = np.array(avg_dUp)
    df['avg_dDown'] = np.array(avg_dDown)
    df['RS'] = df['avg_dUp']/df['avg_dDown']
    df['RSI'] = df['RS'].apply(lambda x: x/(1+x) * 100)
        
    return df

In [5]:
def ATR(DF, n=14):
    '''
    昨日收盤:close.shift()
    '''
    df = DF.copy()
    df['H-L'] = abs(df['high_d'] - df['low_d'])
    df['H-PC'] = abs(df['high_d'] - df['close_d'].shift())
    df['L-PC'] = abs(df['low_d'] - df['close_d'].shift())
    df['TR'] = df[['H-L', 'H-PC', 'L-PC']].max(axis =1, skipna =False)
    df['ATR'] = df['TR'].ewm(span =n, min_periods=n).mean()
    
    return df

In [6]:
def KD(DF, n = 14):
    '''
    function to calculate KD
    '''
    df = DF.copy()
    df['High_14D'] = df['high_d'].rolling(n).max()
    df['Low_14D'] = df['low_d'].rolling(n).min()
    df['RSV'] = (df['close_d'] - df['Low_14D']) / (df['High_14D'] - df['Low_14D']) * 100
    df = df.dropna()    
    df['K'] = np.zeros(len(df))
    df['D'] = np.zeros(len(df))
    
    for i in range(len(df)):
        if i == 0:
            df['K'][i] = 50
            df['D'][i] = 50
        else:
            df['K'][i] = df['K'][i-1]*(2/3) + df['RSV'][i]*(1/3)
            df['D'][i] = df['D'][i-1]*(2/3) + df['K'][i]*(1/3)
    
    return df

# 績效計算function

將交易策略紀錄、與大盤資訊整合在一起，並計算報酬率及其他資訊

In [19]:
def performance_cal(principal, data, trade_book_, market):
  cash = principal
  data_ = data.copy()
  data_ = data_.merge(trade_book_, on = 'mdate', how = 'outer').set_index('mdate')
  data_ = data_.merge(market, on = 'mdate', how = 'inner').set_index('mdate')

  # fillna after merge
  data_['CashValue'].fillna(method = 'ffill', inplace=True)
  data_['CashValue'].fillna(cash, inplace = True)
  data_['TradeUnit'].fillna(0, inplace = True)
  data_['HoldingPosition'] = data_['TradeUnit'].cumsum()

  # Calc strategy value and return
  data_["StockValue"] = [data_['open_d'][i] * data_['HoldingPosition'][i] *1000 for i in range(len(data_.index))]
  data_['TotalValue'] = data_['CashValue'] + data_['StockValue']
  data_['DailyValueChange'] = np.log(data_['TotalValue']) - np.log(data_['TotalValue']).shift(1)
  data_['AccDailyReturn'] =  (data_['TotalValue']/cash - 1) *100

  # Calc BuyHold return
  data_['AccBHReturn'] = (data_['open_d']/data_['open_d'][0] -1) * 100

  # Calc market return
  data_['AccMarketReturn'] = (data_['close_m'] / data_['close_m'][0] - 1) *100

  return data_

def num_output(principal, data_):
  cash = principal
  # Calc numerical output
  overallreturn = round((data_['TotalValue'][-1] / cash - 1) *100, 4) # 總績效
  num_buy, num_sell = len([i for i in data_.BuyOrSell if i == "Buy"]), len([i for i in data_.BuyOrSell if i == "Sell"]) # 買入次數與賣出次數
  num_trade = num_buy + num_sell #交易次數

  avg_hold_period, avg_return = [], []
  tmp_period, tmp_return = [], []
  for i in range(len(trade_book_['mdate'])):
    if trade_book_['BuyOrSell'][i] == 'Buy':
        tmp_period.append(trade_book_["mdate"][i])
        tmp_return.append(trade_book_['CashFlow'][i])
    else:
        sell_date = trade_book_["mdate"][i]
        sell_price = trade_book_['CashFlow'][i] / len(tmp_return)
        avg_hold_period += [sell_date - j for j in tmp_period]
        avg_return += [ abs(sell_price/j) -1  for j in tmp_return]
        tmp_period, tmp_return = [], []

  avg_hold_period_, avg_return_ = np.mean(avg_hold_period), round(np.mean(avg_return) * 100,4) #平均持有期間，平均報酬
  max_win, max_loss = round(max(avg_return)*100, 4) , round(min(avg_return)*100, 4) # 最大獲利報酬，最大損失報酬
  winning_rate = round(len([i for i in avg_return if i > 0]) / len(avg_return) *100, 4)#勝率
  min_cash = round(min(data_['CashValue']),4) #最小現金持有量

  print(' 總績效:', overallreturn, '%\n', 
        '交易次數:', num_trade, '次\n',
        '買入次數:', num_buy, '次\n',
        '賣出次數:', num_sell, '次\n',
        '平均交易報酬:', avg_return_, '%\n',
        '平均持有期間:', avg_hold_period_, '\n',
        '勝率:', winning_rate, '%\n',
        '最大獲利交易報酬:', max_win, '%\n',
        '最大損失交易報酬:', max_loss, '%\n',
        '最低現金持有量:', min_cash )

# 回測應用

In [8]:
## 整理trade book data
def organ_tradebook(trade_book):
  trade_book_ = trade_book.copy()
  trade_book_['mdate'] = [trade_book.BuyTime[i] if trade_book.BuyTime[i] != 0 else trade_book.SellTime[i] for i in trade_book.index]
  trade_book_ = trade_book_.loc[:, ['BuyOrSell', 'CashFlow', 'TradeUnit', 'HoldingPosition', 'CashValue' ,'mdate']]

  return trade_book_

## [MACD](https://medium.com/tej-api-%E9%87%91%E8%9E%8D%E8%B3%87%E6%96%99%E5%88%86%E6%9E%90/%E9%87%8F%E5%8C%96%E5%88%86%E6%9E%90-%E5%85%AB-macd%E6%8C%87%E6%A8%99%E5%9B%9E%E6%B8%AC%E5%AF%A6%E6%88%B0-8794ca221029)
---
當快線(DIF)由下而上穿越慢線(MACD)時，代表股價有上漲的動能存在；反之快線(DIF)向下跌破慢線(MACD)時，代表股價下跌的機率相對高。



In [13]:
data = MACD(stock, a = 12, b =26, c =9)

In [14]:
data

,mdate,volume,open_d,high_d,low_d,close_d,EMA_Fast,EMA_Slow,DIF,MACD
0,2020-01-02,176278,85.3,85.3,83.9,84.1,NaN,NaN,NaN,NaN
1,2020-01-03,251176,84.0,84.0,81.6,82.5,NaN,NaN,NaN,NaN
2,2020-01-06,98132,82.5,83.2,82.0,82.6,NaN,NaN,NaN,NaN
3,2020-01-07,221202,82.8,84.6,82.8,82.8,NaN,NaN,NaN,NaN
4,2020-01-08,112091,82.7,83.0,82.1,83.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
803,2023-04-26,1800073,170.0,173.0,168.0,173.0,167.539593,155.252629,12.286964,12.391663
804,2023-04-27,6525306,170.0,171.0,161.0,171.0,167.918117,156.345027,11.573090,12.227949
805,2023-04-28,6677341,172.0,181.5,170.0,176.5,168.546099,157.504654,11.041445,11.990648
806,2023-05-02,3136432,178.5,181.5,174.0,176.0,170.077468,159.059865,11.017603,11.796039


### 交易策略

In [15]:
data = data.set_index('mdate')

def MACD_Strategy(principal , data):

  cash = principal
  position = 0
  order_unit = 0
  trade_book = pd.DataFrame()
  for i in range(data.shape[0] -2):
    
    DIF_1 = data['DIF'][i]
    DIF_2 = data['DIF'][i-1]
    MACD_1 = data['MACD'][i]
    MACD_2 = data['MACD'][i-1]
    n_time = data.index[i + 1]
    n_open = data['open_d'][i + 1]
    
    
    if position == 0: #進場條件
        if DIF_1>MACD_1 and DIF_2<MACD_2 and cash >= n_open*1000: 
            position += 1
            order_time = n_time
            order_price = n_open
            order_unit = 1
            friction_cost = (20 if order_price*1000*0.001425 < 20 else order_price*1000*0.001425)
            total_cost = -1 * order_price * 1000 - friction_cost
            cash += total_cost
            trade_book = trade_book.append(
                pd.Series(
                [stock_no, 'Buy', order_time, 0, total_cost, order_unit, position, cash
                ]), ignore_index = True)
            
    elif position > 0:
        if DIF_1<MACD_1 and DIF_2>MACD_2: # 出場條件
            order_unit = position
            position = 0
            cover_time = n_time
            cover_price = n_open
            friction_cost = (20 if cover_price*order_unit*1000*0.001425 < 20 else cover_price*order_unit*1000*0.001425) + cover_price*order_unit*1000*0.003
            total_cost = cover_price*order_unit*1000-friction_cost
            cash += total_cost
            trade_book = trade_book.append(pd.Series([
                stock_no, 'Sell', 0, cover_time, total_cost, -1*order_unit, position, cash
            ]), ignore_index=True)
            
        elif DIF_1>MACD_1 and DIF_2<MACD_2 and cash >= n_open*1000: 
            order_unit = 1
            order_time = n_time
            order_price = n_open
            position += 1
            friction_cost = (20 if order_price*1000*0.001425 < 20 else order_price*1000*0.001425) 
            total_cost = -1 * order_price * 1000 - friction_cost
            cash += total_cost
            trade_book = trade_book.append(
            pd.Series(
            [
                stock_no, 'Buy', order_time, 0, total_cost, order_unit, position, cash
            ]), ignore_index = True)
            
  if position > 0: # 最後一天平倉
      order_unit = position
      position = 0
      cover_price = data['open_d'][-1]
      cover_time = data.index[-1]
      friction_cost = (20 if cover_price*order_unit*1000*0.001425 < 20 else cover_price*order_unit*1000*0.001425) + cover_price*order_unit*1000*0.003
      cash += cover_price*order_unit*1000-friction_cost
      trade_book = trade_book.append(
      pd.Series(
      [
         stock_no, 'Sell',0, cover_time, cover_price*order_unit*1000-friction_cost, -1*order_unit, position, cash
      ]), ignore_index = True)    
    
  trade_book.columns = ['Coid', 'BuyOrSell', 'BuyTime', 'SellTime', 'CashFlow','TradeUnit', 'HoldingPosition', 'CashValue']

  return trade_book

### 交易紀錄

In [16]:
trade_book = MACD_Strategy(principal , data)
trade_book

<ipython-input-15-3531b104b4f7>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trade_book = trade_book.append(
<ipython-input-15-3531b104b4f7>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trade_book = trade_book.append(pd.Series([
<ipython-input-15-3531b104b4f7>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trade_book = trade_book.append(
<ipython-input-15-3531b104b4f7>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trade_book = trade_book.append(pd.Series([
<ipython-input-15-3531b104b4f7>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instea

,Coid,BuyOrSell,BuyTime,SellTime,CashFlow,TradeUnit,HoldingPosition,CashValue
0,2731,Buy,2020-04-01 00:00:00,0,-55679.2300,1,1,444320.7700
1,2731,Sell,0,2020-05-18 00:00:00,67997.7725,-1,0,512318.5425
2,2731,Buy,2020-05-27 00:00:00,0,-80114.0000,1,1,432204.5425
3,2731,Sell,0,2020-06-15 00:00:00,77853.9650,-1,0,510058.5075
4,2731,Buy,2020-08-13 00:00:00,0,-67195.6175,1,1,442862.8900
5,2731,Sell,0,2020-08-26 00:00:00,64712.3750,-1,0,507575.2650
6,2731,Buy,2020-08-28 00:00:00,0,-65092.6250,1,1,442482.6400
7,2731,Sell,0,2020-09-30 00:00:00,66603.9675,-1,0,509086.6075
8,2731,Buy,2020-10-05 00:00:00,0,-67896.6150,1,1,441189.9925
9,2731,Sell,0,2020-10-21 00:00:00,65907.0650,-1,0,507097.0575


In [17]:
trade_book_ = organ_tradebook(trade_book)

### 績效計算

In [21]:
data_1 = performance_cal(principal, data, trade_book_, market)
num_output(principal, data_1)

 總績效: 17.9167 %
 交易次數: 52 次
 買入次數: 26 次
 賣出次數: 26 次
 平均交易報酬: 3.9425 %
 平均持有期間: 23 days 02:46:09.230769230 
 勝率: 53.8462 %
 最大獲利交易報酬: 37.2219 %
 最大損失交易報酬: -6.9276 %
 最低現金持有量: 420335.5375


### 繪圖

In [22]:
#累積報酬圖

fig = go.Figure()
fig.add_trace(go.Scatter(
    x = data_1.index, y = data_1.AccDailyReturn, mode = 'lines', name = '交易策略'
))
fig.add_trace(go.Scatter(
    x = data_1.index, y = data_1.AccBHReturn, mode = 'lines', name = '買進持有'
))
fig.add_trace(go.Scatter(
    x = data_1.index, y = data_1.AccMarketReturn, mode = 'lines', name = '市場大盤'
))
fig.update_layout(
    title = stock_no + '累積報酬圖', yaxis_title = '累積報酬(%)', xaxis_title = '時間'
)
fig.show()

In [24]:
#買賣點視覺化

graph_price = go.Scatter(x = data_1.index,
                         y = data_1['close_d'],                  
                         mode = 'lines', line_color = 'grey', 
                         name='股價')

graph_dif = go.Scatter(x = data_1.index,
                         y = data_1['DIF'],                  
                         mode = 'lines',  
                         name='DIF快線')

graph_macd = go.Scatter(x = data_1.index,
                         y = data_1['MACD'],                  
                         mode = 'lines',  
                         name='MACD慢線')

graph_buy= go.Scatter(x=data_1.index, y= np.where(data_1['TradeUnit'] > 0, data_1['open_d'], np.nan), 
                            mode='markers', marker_symbol="triangle-up", marker_color="red", marker_size=15,
                            name='Buy', text = "買入: "+ data_1['TradeUnit'].apply(str) + '張')
        
graph_sell= go.Scatter(x=data_1.index, y= np.where(data_1['TradeUnit'] < 0, data_1['open_d'], np.nan), 
                            mode='markers', marker_symbol="triangle-down", marker_color="limegreen", marker_size=15,
                            name='Sell', text = "賣出: "+ abs(data_1['TradeUnit']).apply(str) + '張')


#資料
fig = subplots.make_subplots(subplot_titles = ('買賣點觀察',), specs=[[{"secondary_y": True}]])
fig.add_trace(graph_price,  secondary_y=False)
fig.add_trace(graph_dif, secondary_y=True)
fig.add_trace(graph_macd, secondary_y=True)
fig.add_trace(graph_buy,  secondary_y=False)
fig.add_trace(graph_sell,  secondary_y=False)

#x y軸設定
fig.update_xaxes(title_text="日期")
fig.update_yaxes(title_text="股價", secondary_y=False)
fig.update_yaxes(title_text="快慢線", secondary_y=True)

fig.show()


## [布林通道](https://medium.com/tej-api-%E9%87%91%E8%9E%8D%E8%B3%87%E6%96%99%E5%88%86%E6%9E%90/%E5%AF%A6%E6%88%B0%E6%87%89%E7%94%A8-%E5%B8%83%E6%9E%97%E9%80%9A%E9%81%93%E4%BA%A4%E6%98%93%E7%AD%96%E7%95%A5-737b65faac2f)


---



當收盤價觸碰到上界時，視為接下來可能會下跌的訊號，以隔日開盤價拋售持有部位。

當收盤價觸碰到下界時，視為接下來有可能谷底反彈的訊號，以隔日開盤價買入一單位。當滿足上述條件時，以及滿足本金充足、已持有部位與當日收盤價低於上次買入訊號收盤價時，則繼續加碼一單位。

In [25]:
data = BollingerBand(stock)
data = data.set_index('mdate')

In [26]:
data

,volume,open_d,high_d,low_d,close_d,SMA,BBu,BBd,BB_width
mdate,,,,,,,,,
2020-01-02,176278,85.3,85.3,83.9,84.1,NaN,NaN,NaN,NaN
2020-01-03,251176,84.0,84.0,81.6,82.5,NaN,NaN,NaN,NaN
2020-01-06,98132,82.5,83.2,82.0,82.6,NaN,NaN,NaN,NaN
2020-01-07,221202,82.8,84.6,82.8,82.8,NaN,NaN,NaN,NaN
2020-01-08,112091,82.7,83.0,82.1,83.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2023-04-26,1800073,170.0,173.0,168.0,173.0,160.150,190.430522,129.869478,60.561044
2023-04-27,6525306,170.0,171.0,161.0,171.0,162.125,189.705564,134.544436,55.161128
2023-04-28,6677341,172.0,181.5,170.0,176.5,164.025,190.023990,138.026010,51.997981


In [27]:
fig = px.line(data,   
            x=data.index, 
            y=["close_d","BBu","BBd"], 
            color_discrete_sequence = px.colors.qualitative.Vivid
            )
fig.show()

### 交易策略

逆勢操作:\
若收盤價觸碰BBU20下界，以隔日開盤價買入；若收盤價觸碰BBU20上界，以隔日開盤價賣出。\
若已持有部位，上述條件滿足且本金充分時，則多買入一張

In [28]:
def BollingerBand_Strategy(principal , data):

  cash = principal
  position = 0
  order_unit = 0
  trade_book = pd.DataFrame()

  for i in range(data.shape[0] -2):
    
    cu_time = data.index[i]
    cu_close = data.loc[cu_time, 'close_d']
    cu_bbl, cu_bbu = data.loc[cu_time, 'BBd'], data.loc[cu_time, 'BBu']
    n_time = data.index[i + 1]
    n_open = data['open_d'][i + 1]
    
    
    if position == 0: #進場條件
        if cu_close <= cu_bbl and cash >= n_open*1000: 
            position += 1
            order_time = n_time
            order_price = n_open
            order_unit = 1
            friction_cost = (20 if order_price*1000*0.001425 < 20 else order_price*1000*0.001425)
            total_cost = -1 * order_price * 1000 - friction_cost
            cash += total_cost
            trade_book = trade_book.append(
                pd.Series(
                [stock_no, 'Buy', order_time, 0, total_cost, order_unit, position, cash
                ]), ignore_index = True)
            
    elif position > 0:
        if cu_close >= cu_bbu: # 出場條件
            order_unit = position
            position = 0
            cover_time = n_time
            cover_price = n_open
            friction_cost = (20 if cover_price*order_unit*1000*0.001425 < 20 else cover_price*order_unit*1000*0.001425) + cover_price*order_unit*1000*0.003
            total_cost = cover_price*order_unit*1000-friction_cost
            cash += total_cost
            trade_book = trade_book.append(pd.Series([
                stock_no, 'Sell', 0, cover_time, total_cost, -1*order_unit, position, cash
            ]), ignore_index=True)
            
        elif cu_close <= cu_bbl and cu_close <= order_price and cash >= n_open*1000: #加碼條件: 碰到下界，比過去買入價格貴
            order_unit = 1
            order_time = n_time
            order_price = n_open
            position += 1
            friction_cost = (20 if order_price*1000*0.001425 < 20 else order_price*1000*0.001425) 
            total_cost = -1 * order_price * 1000 - friction_cost
            cash += total_cost
            trade_book = trade_book.append(
            pd.Series(
            [
                stock_no, 'Buy', order_time, 0, total_cost, order_unit, position, cash
            ]), ignore_index = True)
            
  if position > 0: # 最後一天平倉
      order_unit = position
      position = 0
      cover_price = data['open_d'][-1]
      cover_time = data.index[-1]
      friction_cost = (20 if cover_price*order_unit*1000*0.001425 < 20 else cover_price*order_unit*1000*0.001425) + cover_price*order_unit*1000*0.003
      cash += cover_price*order_unit*1000-friction_cost
      trade_book = trade_book.append(
      pd.Series(
      [
          stock_no, 'Sell',0, cover_time, cover_price*order_unit*1000-friction_cost, -1*order_unit, position, cash
      ]), ignore_index = True)    
    
  trade_book.columns = ['Coid', 'BuyOrSell', 'BuyTime', 'SellTime', 'CashFlow','TradeUnit', 'HoldingPosition', 'CashValue']

  return trade_book

### 交易紀錄

In [29]:
trade_book = BollingerBand_Strategy(principal , data)
trade_book

<ipython-input-28-dbc332ff35ac>:26: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-28-dbc332ff35ac>:52: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-28-dbc332ff35ac>:52: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-28-dbc332ff35ac>:52: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-28-dbc332ff35ac>:52: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-28-dbc332ff35ac>:52: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Us

,Coid,BuyOrSell,BuyTime,SellTime,CashFlow,TradeUnit,HoldingPosition,CashValue
0,2731,Buy,2020-03-11 00:00:00,0,-69198.4675,1,1,430801.5325
1,2731,Buy,2020-03-13 00:00:00,0,-63991.0575,1,2,366810.4750
2,2731,Buy,2020-03-17 00:00:00,0,-56981.0825,1,3,309829.3925
3,2731,Buy,2020-03-18 00:00:00,0,-55278.6600,1,4,254550.7325
4,2731,Buy,2020-03-19 00:00:00,0,-47868.1150,1,5,206682.6175
5,2731,Buy,2020-03-20 00:00:00,0,-46165.6925,1,6,160516.9250
6,2731,Sell,0,2020-04-16 00:00:00,411570.7050,-6,0,572087.6300
7,2731,Buy,2020-06-30 00:00:00,0,-77109.7250,1,1,494977.9050
8,2731,Buy,2020-07-08 00:00:00,0,-74305.7350,1,2,420672.1700
9,2731,Buy,2020-07-09 00:00:00,0,-73104.0250,1,3,347568.1450


In [30]:
trade_book_ = organ_tradebook(trade_book)

### 績效計算

In [31]:
data_2 = performance_cal(principal, data, trade_book_, market)
num_output(principal, data_2)

 總績效: 51.6745 %
 交易次數: 51 次
 買入次數: 40 次
 賣出次數: 11 次
 平均交易報酬: 9.3324 %
 平均持有期間: 37 days 22:48:00 
 勝率: 82.5 %
 最大獲利交易報酬: 48.5846 %
 最大損失交易報酬: -12.4624 %
 最低現金持有量: 78284.9625


In [32]:
data_2

,volume,open_d,high_d,low_d,close_d,SMA,BBu,BBd,BB_width,BuyOrSell,...,open_m,high_m,low_m,close_m,StockValue,TotalValue,DailyValueChange,AccDailyReturn,AccBHReturn,AccMarketReturn
mdate,,,,,,,,,,,,,,,,,,,,,
2020-01-02,176278,85.3,85.3,83.9,84.1,NaN,NaN,NaN,NaN,NaN,...,12026.50,12110.74,12026.23,12100.48,0.0,500000.00,NaN,0.000000,0.000000,0.000000
2020-01-03,251176,84.0,84.0,81.6,82.5,NaN,NaN,NaN,NaN,NaN,...,12167.44,12197.64,12023.60,12110.43,0.0,500000.00,0.0,0.000000,-1.524033,0.082228
2020-01-06,98132,82.5,83.2,82.0,82.6,NaN,NaN,NaN,NaN,NaN,...,12035.71,12040.08,11953.36,11953.36,0.0,500000.00,0.0,0.000000,-3.282532,-1.215820
2020-01-07,221202,82.8,84.6,82.8,82.8,NaN,NaN,NaN,NaN,NaN,...,11961.97,11986.03,11822.40,11880.32,0.0,500000.00,0.0,0.000000,-2.930832,-1.819432
2020-01-08,112091,82.7,83.0,82.1,83.0,NaN,NaN,NaN,NaN,NaN,...,11818.76,11899.67,11777.45,11817.10,0.0,500000.00,0.0,0.000000,-3.048066,-2.341891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-26,1800073,170.0,173.0,168.0,173.0,160.150,190.430522,129.869478,60.561044,NaN,...,15352.03,15398.40,15284.46,15374.63,0.0,758372.71,0.0,51.674542,99.296600,27.058018
2023-04-27,6525306,170.0,171.0,161.0,171.0,162.125,189.705564,134.544436,55.161128,NaN,...,15374.39,15455.88,15306.69,15411.49,0.0,758372.71,0.0,51.674542,99.296600,27.362634
2023-04-28,6677341,172.0,181.5,170.0,176.5,164.025,190.023990,138.026010,51.997981,NaN,...,15477.34,15580.63,15477.34,15579.18,0.0,758372.71,0.0,51.674542,101.641266,28.748446


### 繪圖

In [ ]:
#累積報酬圖

fig = go.Figure()
fig.add_trace(go.Scatter(
    x = data_2.index, y = data_2.AccDailyReturn, mode = 'lines', name = '交易策略'
))
fig.add_trace(go.Scatter(
    x = data_2.index, y = data_2.AccBHReturn, mode = 'lines', name = '買進持有'
))
fig.add_trace(go.Scatter(
    x = data_2.index, y = data_2.AccMarketReturn, mode = 'lines', name = '市場大盤'
))
fig.update_layout(
    title = stock_no + '累積報酬圖', yaxis_title = '累積報酬(%)', xaxis_title = '時間'
)
fig.show()

In [ ]:
# 買賣訊號繪製圖
Buy_index = [i-1 for i in range(len(data_2['BuyOrSell'])) if data_2['BuyOrSell'][i] == 'Buy']
Sell_index = [i-1 for i in range(len(data_2['BuyOrSell'])) if data_2['BuyOrSell'][i] == 'Sell']
Buy_point, Sell_point = data_2.iloc[Buy_index, :], data_2.iloc[Sell_index, :]
fig = go.Figure()
fig.add_trace(go.Scatter(x = data_2.index,
                        y = data_2.close_d,
                        mode = 'lines', 
                        name = '收盤價格'))
fig.add_trace(
    go.Scatter(
        x = data_2.index, y = data_2.BBu, mode = 'lines',name = '上界'
    )
)
fig.add_trace(
    go.Scatter(
        x = data_2.index, y = data_2.BBd, mode = 'lines',name = '下界'
    )
)

fig.add_trace(
    go.Scatter(
        x = Buy_point.index, y = Buy_point.close_d, mode = 'markers',
        marker_symbol="triangle-up", marker_color="red", marker_size=12, name = '買入訊號'
    )
)
fig.add_trace(
    go.Scatter(
        x = Sell_point.index, y = Sell_point.close_d, mode = 'markers',
        marker_symbol="triangle-up", marker_color="limegreen", marker_size=12, name = '賣出訊號'
    )
)

fig.add_vrect(
    x0="2021-11-02", x1="2021-12-20",
    fillcolor="lightBlue", opacity=0.5,
    layer="below", line_width=0,
)

fig.add_vrect(
    x0="2022-03-23", x1="2022-08-05",
    fillcolor="LightGreen", opacity=0.5,
    layer="below", line_width=0,
)

fig.update_layout(
    title = stock_no + '股價走勢圖', yaxis_title = '股票價格', xaxis_title = '時間',
)

fig.show()

##[RSI指標策略](https://medium.com/tej-api-%E9%87%91%E8%9E%8D%E8%B3%87%E6%96%99%E5%88%86%E6%9E%90/%E9%87%8F%E5%8C%96%E5%88%86%E6%9E%90-%E5%8D%81%E4%B8%89-rsi%E6%8C%87%E6%A8%99%E5%9B%9E%E6%B8%AC%E5%AF%A6%E6%88%B0-1d915873f1f7)
---
RSI指標的判斷依據如下：
1.   RSI < 30為賣超情形，RSI > 70為買超情形，RSI在50之間波動為買賣方力量持平，上述(30,70)組合也有較保守的(20,80)判斷依據。
2.   RSI指標背離，也就是RSI走勢與股價走勢不一致，代表市場將出現反轉，是買賣訊號的一個依據。
3.   黃金交叉，短天期 RSI向上突破長天期 RSI，代表市場即將進入多頭。
4.   死亡交叉，短天期 RSI向下突破長天期 RSI，代表市場即將進入空頭。





In [33]:
data_14 = RSI(stock,14)
data_7 = RSI(stock,7)
data = data_14.merge(data_7, on = ['mdate','open_d','high_d','low_d','close_d','volume','daliy_change','dUp','dDown'])

In [34]:
data.rename(columns={'avg_dUp_x':'avg_dUp14', 'avg_dDown_x':'avg_dDown14', 
                        'RS_x':'RS14', 'RSI_x':'RSI14',
                        'avg_dUp_y':'avg_dUp7', 'avg_dDown_y':'avg_dDown7', 
                        'RS_y':'RS7', 'RSI_y':'RSI7',}, inplace = True)
data

,mdate,volume,open_d,high_d,low_d,close_d,daliy_change,dUp,dDown,avg_dUp14,avg_dDown14,RS14,RSI14,avg_dUp7,avg_dDown7,RS7,RSI7
0,2020-01-02,176278,85.3,85.3,83.9,84.1,NaN,0.0,0.0,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN
1,2020-01-03,251176,84.0,84.0,81.6,82.5,-1.6,0.0,1.6,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN
2,2020-01-06,98132,82.5,83.2,82.0,82.6,0.1,0.1,0.0,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN
3,2020-01-07,221202,82.8,84.6,82.8,82.8,0.2,0.2,0.0,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN
4,2020-01-08,112091,82.7,83.0,82.1,83.0,0.2,0.2,0.0,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
803,2023-04-26,1800073,170.0,173.0,168.0,173.0,2.0,2.0,0.0,2.546198,1.167948,2.180061,68.554064,2.339692,1.335901,1.751396,63.654816
804,2023-04-27,6525306,170.0,171.0,161.0,171.0,-2.0,0.0,2.0,2.364326,1.227380,1.926320,65.827383,2.005451,1.430773,1.401656,58.362060
805,2023-04-28,6677341,172.0,181.5,170.0,176.5,5.5,5.5,0.0,2.588303,1.139710,2.271019,69.428485,2.504672,1.226376,2.042335,67.130513
806,2023-05-02,3136432,178.5,181.5,174.0,176.0,-0.5,0.0,0.5,2.403424,1.094016,2.196881,68.719513,2.146862,1.122608,1.912387,65.663904


In [35]:
data = data.reset_index()

fig = go.Figure()


fig.add_trace(go.Scatter(x = data['mdate'], y = data['RSI14'], name = "14-RSI")) 
fig.add_trace(go.Scatter(x = data['mdate'], y = data['RSI7'], name = "7-RSI"))
fig.add_trace(go.Scatter(x = data['mdate'], y = np.full(shape=800, fill_value = 70), line = dict(color='grey', width=3, dash='dash'), name = '70RSI'))                       
fig.add_trace(go.Scatter(x = data['mdate'], y = np.full(shape=800, fill_value = 30), line = dict(color='grey', width=3, dash='dash'), name = '30RSI'))                       

graph_gold = go.Scatter(x=data['mdate'], y= np.where((data['RSI7'].shift() < data['RSI14'].shift()) & (data['RSI7'] >= data['RSI14']) & (data['RSI14'] < 30),
                                                        data['RSI14'], np.nan), 
                            mode='markers', marker_symbol="x", marker_color="gold", marker_size=15, name='黃金交叉')

graph_dead = go.Scatter(x=data["mdate"], y= np.where((data['RSI7'].shift() > data['RSI14'].shift()) & (data['RSI7'] <= data['RSI14']) & (data['RSI14'] > 70), 
                                                        data['RSI14'], np.nan), 
                            mode='markers', marker_symbol="x", marker_color="black", marker_size=10, name='死亡交叉')

fig.add_trace(graph_gold)
fig.add_trace(graph_dead)

fig.update_layout(title='RSI走勢圖',
                 xaxis_title="日期",
                 yaxis_title="RSI")

fig.show()

### 交易策略

In [36]:
data = data.set_index('mdate')
# 交易訊號只發生在買賣超階段與黃金交叉或死亡交叉同時發生的情境下

def RSI_Strategy(principal , data):
  cash = principal
  position = 0
  trade = 0
  order_unit = 0
  trade_book = pd.DataFrame()

  for i in range(data.shape[0] -2):
    
    RSI_14 = data['RSI14'][i]
    RSI_14_y = data['RSI14'][i-1]
    RSI_7 = data['RSI7'][i]
    RSI_7_y = data['RSI7'][i-1]
    n_time = data.index[i + 1]
    n_open = data['open_d'][i + 1]
    
    
    if position == 0: #進場條件
        if RSI_14<=30 and RSI_14_y>RSI_7_y and RSI_14<=RSI_7 and trade==0 and cash >= n_open*1000: 
            position += 1
            order_time = n_time
            order_price = n_open
            order_unit = 1
            friction_cost = (20 if order_price*1000*0.001425 < 20 else order_price*1000*0.001425)
            total_cost = -1 * order_price * 1000 - friction_cost
            cash += total_cost
            trade_book = trade_book.append(
                pd.Series(
                [stock_no, 'Buy', order_time, 0, total_cost, order_unit, position, cash
                ]), ignore_index = True)
            
    elif position > 0:
        if RSI_14>=70 and RSI_14_y<RSI_7_y and RSI_14>=RSI_7 and trade==1: # 出場條件
            order_unit = position
            position = 0
            cover_time = n_time
            cover_price = n_open
            friction_cost = (20 if cover_price*order_unit*1000*0.001425 < 20 else cover_price*order_unit*1000*0.001425) + cover_price*order_unit*1000*0.003
            total_cost = cover_price*order_unit*1000-friction_cost
            cash += total_cost
            trade_book = trade_book.append(pd.Series([
                stock_no, 'Sell', 0, cover_time, total_cost, -1*order_unit, position, cash
            ]), ignore_index=True)
            
        elif RSI_14<=30 and RSI_14_y>RSI_7_y and RSI_14<=RSI_7 and trade==0 and cash >= n_open*1000: 
            order_unit = 1
            order_time = n_time
            order_price = n_open
            position += 1
            friction_cost = (20 if order_price*1000*0.001425 < 20 else order_price*1000*0.001425) 
            total_cost = -1 * order_price * 1000 - friction_cost
            cash += total_cost
            trade_book = trade_book.append(
            pd.Series(
            [
                stock_no, 'Buy', order_time, 0, total_cost, order_unit, position, cash
            ]), ignore_index = True)
            
  if position > 0: # 最後一天平倉
      order_unit = position
      position = 0
      cover_price = data['open_d'][-1]
      cover_time = data.index[-1]
      friction_cost = (20 if cover_price*order_unit*1000*0.001425 < 20 else cover_price*order_unit*1000*0.001425) + cover_price*order_unit*1000*0.003
      cash += cover_price*order_unit*1000-friction_cost
      trade_book = trade_book.append(
      pd.Series(
      [
          stock_no, 'Sell',0, cover_time, cover_price*order_unit*1000-friction_cost, -1*order_unit, position, cash
      ]), ignore_index = True)    
    
  trade_book.columns = ['Coid', 'BuyOrSell', 'BuyTime', 'SellTime', 'CashFlow','TradeUnit', 'HoldingPosition', 'CashValue']

  return trade_book

### 交易紀錄

In [37]:
trade_book = RSI_Strategy(principal , data)
trade_book

<ipython-input-36-90c6498a6a2c>:30: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-36-90c6498a6a2c>:56: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-36-90c6498a6a2c>:56: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-36-90c6498a6a2c>:69: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



,Coid,BuyOrSell,BuyTime,SellTime,CashFlow,TradeUnit,HoldingPosition,CashValue
0,2731,Buy,2020-02-05 00:00:00,0,-70600.4625,1,1,429399.5375
1,2731,Buy,2020-03-23 00:00:00,0,-47066.9750,1,2,382332.5625
2,2731,Buy,2020-03-25 00:00:00,0,-51272.9600,1,3,331059.6025
3,2731,Sell,0,2023-05-03 00:00:00,522676.8750,-3,0,853736.4775


In [38]:
trade_book_ = organ_tradebook(trade_book)

### 績效試算

In [39]:
data_3 = performance_cal(principal, data, trade_book_, market)
num_output(principal, data_3)

 總績效: 70.7473 %
 交易次數: 4 次
 買入次數: 3 次
 賣出次數: 1 次
 平均交易報酬: 218.9141 %
 平均持有期間: 1151 days 00:00:00 
 勝率: 100.0 %
 最大獲利交易報酬: 270.1653 %
 最大損失交易報酬: 146.7769 %
 最低現金持有量: 331059.6025


In [40]:
data_3

,index,volume,open_d,high_d,low_d,close_d,daliy_change,dUp,dDown,avg_dUp14,...,open_m,high_m,low_m,close_m,StockValue,TotalValue,DailyValueChange,AccDailyReturn,AccBHReturn,AccMarketReturn
mdate,,,,,,,,,,,,,,,,,,,,,
2020-01-02,0,176278,85.3,85.3,83.9,84.1,NaN,0.0,0.0,0.000000,...,12026.50,12110.74,12026.23,12100.48,0.0,500000.0000,NaN,0.000000,0.000000,0.000000
2020-01-03,1,251176,84.0,84.0,81.6,82.5,-1.6,0.0,1.6,0.000000,...,12167.44,12197.64,12023.60,12110.43,0.0,500000.0000,0.000000,0.000000,-1.524033,0.082228
2020-01-06,2,98132,82.5,83.2,82.0,82.6,0.1,0.1,0.0,0.000000,...,12035.71,12040.08,11953.36,11953.36,0.0,500000.0000,0.000000,0.000000,-3.282532,-1.215820
2020-01-07,3,221202,82.8,84.6,82.8,82.8,0.2,0.2,0.0,0.000000,...,11961.97,11986.03,11822.40,11880.32,0.0,500000.0000,0.000000,0.000000,-2.930832,-1.819432
2020-01-08,4,112091,82.7,83.0,82.1,83.0,0.2,0.2,0.0,0.000000,...,11818.76,11899.67,11777.45,11817.10,0.0,500000.0000,0.000000,0.000000,-3.048066,-2.341891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-26,803,1800073,170.0,173.0,168.0,173.0,2.0,2.0,0.0,2.546198,...,15352.03,15398.40,15284.46,15374.63,510000.0,841059.6025,-0.005336,68.211921,99.296600,27.058018
2023-04-27,804,6525306,170.0,171.0,161.0,171.0,-2.0,0.0,2.0,2.364326,...,15374.39,15455.88,15306.69,15411.49,510000.0,841059.6025,0.000000,68.211921,99.296600,27.362634
2023-04-28,805,6677341,172.0,181.5,170.0,176.5,5.5,5.5,0.0,2.588303,...,15477.34,15580.63,15477.34,15579.18,516000.0,847059.6025,0.007109,69.411921,101.641266,28.748446


### 繪圖

In [ ]:
#累積報酬圖

fig = go.Figure()
fig.add_trace(go.Scatter(
    x = data_3.index, y = data_3.AccDailyReturn, mode = 'lines', name = '交易策略'
))
fig.add_trace(go.Scatter(
    x = data_3.index, y = data_3.AccBHReturn, mode = 'lines',  name = '買進持有'
))
fig.add_trace(go.Scatter(
    x = data_3.index, y = data_3.AccMarketReturn, mode = 'lines', name = '市場大盤'
))
fig.update_layout(
    title = stock_no + '累積報酬圖', yaxis_title = '累積報酬(%)', xaxis_title = '時間'
)
fig.show()

In [ ]:
# 買賣訊號繪製圖
Buy_index = [i-1 for i in range(len(data_3['BuyOrSell'])) if data_3['BuyOrSell'][i] == 'Buy']
Sell_index = [i-1 for i in range(len(data_3['BuyOrSell'])) if data_3['BuyOrSell'][i] == 'Sell']
Buy_point, Sell_point = data_3.iloc[Buy_index, :], data_3.iloc[Sell_index, :]
fig = go.Figure()
fig.add_trace(go.Scatter(x = data_3.index,
                        y = data_3.close_d,
                        mode = 'lines', 
                        name = '收盤價格'))

fig.add_trace(
    go.Scatter(
        x = Buy_point.index, y = Buy_point.close_d, mode = 'markers',
        marker_symbol="triangle-up", marker_color="red", marker_size=12, name = '買入訊號'
    )
)
fig.add_trace(
    go.Scatter(
        x = Sell_point.index, y = Sell_point.close_d, mode = 'markers',
        marker_symbol="triangle-up", marker_color="limegreen", marker_size=12, name = '賣出訊號'
    )
)

fig.add_vrect(
    x0="2021-11-02", x1="2021-12-20",
    fillcolor="lightBlue", opacity=0.5,
    layer="below", line_width=0,
)

fig.add_vrect(
    x0="2022-03-23", x1="2022-08-05",
    fillcolor="LightGreen", opacity=0.5,
    layer="below", line_width=0,
)

fig.update_layout(
    title = stock_no + '股價走勢圖', yaxis_title = '股票價格', xaxis_title = '時間',
)

fig.show()

## 報酬率比較

In [ ]:
#累積報酬圖

fig = go.Figure()
fig.add_trace(go.Scatter(
    x = data_1.index, y = data_1.AccDailyReturn, mode = 'lines', name = 'MACD交易策略'
))
fig.add_trace(go.Scatter(
    x = data_1.index, y = data_2.AccDailyReturn, mode = 'lines', name = '布林通道交易策略'
))
fig.add_trace(go.Scatter(
    x = data_1.index, y = data_3.AccDailyReturn, mode = 'lines', name = 'RSI交易策略'
))
fig.add_trace(go.Scatter(
    x = data_1.index, y = data_1.AccBHReturn, mode = 'lines', line = dict(shape = 'linear', color = 'rgb(10, 120, 24)', width=2,  dash = 'dot'), 
    name = '買進持有', connectgaps = True
))
fig.add_trace(go.Scatter(
    x = data_1.index, y = data_1.AccMarketReturn, mode = 'lines', line = dict(shape = 'linear', color = 'rgb(100, 10, 100)', width = 2, dash = 'dot'),
     name = '市場大盤', connectgaps = True
))
fig.update_layout(
    title = stock_no + '累積報酬圖', yaxis_title = '累積報酬(%)', xaxis_title = '時間'
)
fig.show()

# Compare

---

我們選擇兩種策略 去回測三種不同的股票

In [73]:
stock_no1 = '2324'
stock_no2 = '2303'
stock_no3 = '1733'
start, end ='2020-01-01', '2023-05-03'
principal = 500000 ##本金

D1 = DataLoader()
stock1 = D1.taiwan_stock_daily(stock_id=stock_no1, start_date=start, end_date=end)
stock1.rename(columns={'date':'mdate', 'open':'open_d', 
                        'max':'high_d', 'min':'low_d',
                        'close':'close_d', 'Trading_Volume':'volume', }, inplace = True)
stock1.drop(columns=['stock_id','Trading_money','spread','Trading_turnover'], inplace = True)
stock1['mdate'] = pd.to_datetime(stock1['mdate']) # 轉換日期格式('文字'轉'時間')

D2 = DataLoader()
stock2 = D2.taiwan_stock_daily(stock_id=stock_no2, start_date=start, end_date=end)
stock2.rename(columns={'date':'mdate', 'open':'open_d', 
                        'max':'high_d', 'min':'low_d',
                        'close':'close_d', 'Trading_Volume':'volume', }, inplace = True)
stock2.drop(columns=['stock_id','Trading_money','spread','Trading_turnover'], inplace = True)
stock2['mdate'] = pd.to_datetime(stock2['mdate']) # 轉換日期格式('文字'轉'時間')

D3 = DataLoader()
stock3 = D3.taiwan_stock_daily(stock_id=stock_no3, start_date=start, end_date=end)
stock3.rename(columns={'date':'mdate', 'open':'open_d', 
                        'max':'high_d', 'min':'low_d',
                        'close':'close_d', 'Trading_Volume':'volume', }, inplace = True)
stock3.drop(columns=['stock_id','Trading_money','spread','Trading_turnover'], inplace = True)
stock3['mdate'] = pd.to_datetime(stock3['mdate']) # 轉換日期格式('文字'轉'時間')

# 大盤
D4 = DataLoader()
market = D4.taiwan_stock_daily(stock_id='TAIEX', start_date=start, end_date=end)
market.rename(columns={'date':'mdate', 'open':'open_m', 
                        'max':'high_m', 'min':'low_m',
                        'close':'close_m', 'Trading_Volume':'volume_m', }, inplace = True)
market.drop(columns=['stock_id','Trading_money','spread','Trading_turnover'], inplace = True)
market['mdate'] = pd.to_datetime(market['mdate'])

## 布林通道策略

In [74]:
data1 = BollingerBand(stock1)
data1 = data1.set_index('mdate')
data2 = BollingerBand(stock2)
data2 = data2.set_index('mdate')
data3 = BollingerBand(stock3)
data3 = data3.set_index('mdate')

trade_book1_B = BollingerBand_Strategy(principal , data1)
trade_book2_B = BollingerBand_Strategy(principal , data2)
trade_book3_B = BollingerBand_Strategy(principal , data3)

trade_book_1B = organ_tradebook(trade_book1_B)
trade_book_2B = organ_tradebook(trade_book2_B)
trade_book_3B = organ_tradebook(trade_book3_B)

data_1B = performance_cal(principal, data1, trade_book_1B, market)
data_2B = performance_cal(principal, data2, trade_book_2B, market)
data_3B = performance_cal(principal, data3, trade_book_3B, market)

<ipython-input-28-dbc332ff35ac>:26: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-28-dbc332ff35ac>:52: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-28-dbc332ff35ac>:52: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-28-dbc332ff35ac>:52: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-28-dbc332ff35ac>:52: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-28-dbc332ff35ac>:52: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Us

In [75]:
#累積報酬圖

fig = go.Figure()
fig.add_trace(go.Scatter(
    x = data_1B.index, y = data_1B.AccDailyReturn, mode = 'lines', name = stock_no1
))
fig.add_trace(go.Scatter(
    x = data_2B.index, y = data_2B.AccDailyReturn, mode = 'lines', name = stock_no2
))
fig.add_trace(go.Scatter(
    x = data_3B.index, y = data_3B.AccDailyReturn, mode = 'lines', name = stock_no3
))
fig.update_layout(
    title = '布林通道策略累積報酬圖', yaxis_title = '累積報酬(%)', xaxis_title = '時間'
)
fig.show()

## RSI 策略

In [76]:
data1_14 = RSI(stock1,14)
data1_7 = RSI(stock1,7)
data1 = data1_14.merge(data1_7, on = ['mdate','open_d','high_d','low_d','close_d','volume','daliy_change','dUp','dDown'])
data1.rename(columns={'avg_dUp_x':'avg_dUp14', 'avg_dDown_x':'avg_dDown14', 
                        'RS_x':'RS14', 'RSI_x':'RSI14',
                        'avg_dUp_y':'avg_dUp7', 'avg_dDown_y':'avg_dDown7', 
                        'RS_y':'RS7', 'RSI_y':'RSI7'}, inplace = True)
data1 = data1.set_index('mdate')

data2_14 = RSI(stock2,14)
data2_7 = RSI(stock2,7)
data2 = data2_14.merge(data2_7, on = ['mdate','open_d','high_d','low_d','close_d','volume','daliy_change','dUp','dDown'])
data2.rename(columns={'avg_dUp_x':'avg_dUp14', 'avg_dDown_x':'avg_dDown14', 
                        'RS_x':'RS14', 'RSI_x':'RSI14',
                        'avg_dUp_y':'avg_dUp7', 'avg_dDown_y':'avg_dDown7', 
                        'RS_y':'RS7', 'RSI_y':'RSI7'}, inplace = True)
data2 = data2.set_index('mdate')

data3_14 = RSI(stock3,14)
data3_7 = RSI(stock3,7)
data3 = data3_14.merge(data3_7, on = ['mdate','open_d','high_d','low_d','close_d','volume','daliy_change','dUp','dDown'])
data3.rename(columns={'avg_dUp_x':'avg_dUp14', 'avg_dDown_x':'avg_dDown14', 
                        'RS_x':'RS14', 'RSI_x':'RSI14',
                        'avg_dUp_y':'avg_dUp7', 'avg_dDown_y':'avg_dDown7', 
                        'RS_y':'RS7', 'RSI_y':'RSI7'}, inplace = True)
data3 = data3.set_index('mdate')

trade_book1_RSI = RSI_Strategy(principal , data1)
trade_book2_RSI = RSI_Strategy(principal , data2)
trade_book3_RSI = RSI_Strategy(principal , data3)

trade_book_1RSI = organ_tradebook(trade_book1_RSI)
trade_book_2RSI = organ_tradebook(trade_book2_RSI)
trade_book_3RSI = organ_tradebook(trade_book3_RSI)

data_1RSI = performance_cal(principal, data1, trade_book_1RSI, market)
data_2RSI = performance_cal(principal, data2, trade_book_2RSI, market)
data_3RSI = performance_cal(principal, data3, trade_book_3RSI, market)

<ipython-input-36-90c6498a6a2c>:30: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-36-90c6498a6a2c>:69: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-36-90c6498a6a2c>:30: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-36-90c6498a6a2c>:69: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-36-90c6498a6a2c>:30: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-36-90c6498a6a2c>:69: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Us

In [77]:
#累積報酬圖

fig = go.Figure()
fig.add_trace(go.Scatter(
    x = data_1RSI.index, y = data_1RSI.AccDailyReturn, mode = 'lines', name = stock_no1
))
fig.add_trace(go.Scatter(
    x = data_2RSI.index, y = data_2RSI.AccDailyReturn, mode = 'lines', name = stock_no2
))
fig.add_trace(go.Scatter(
    x = data_3RSI.index, y = data_3RSI.AccDailyReturn, mode = 'lines', name = stock_no3
))
fig.update_layout(
    title = 'ＲＳＩ策略累積報酬圖', yaxis_title = '累積報酬(%)', xaxis_title = '時間'
)
fig.show()